# Profile hydrologic model


In [1]:
import numpy as np
from numpy.typing import NDArray
import pandas as pd
from pandas import DataFrame, Series
import potions as pt

In [2]:
data_path: str = "../input/Sleepers_Results.txt"
in_df: DataFrame = pd.read_csv(
    data_path, sep="\\s+", index_col="Date", parse_dates=True
)

In [3]:
# model: pt.HbvModel = pt.HbvModel.default()
# model: pt.ThreeLayerModel = pt.ThreeLayerModel.default()
model: pt.HbvLateralModel = pt.HbvLateralModel.default()
init_state: NDArray = model.default_init_state()
dates: Series = in_df.index  # type: ignore
forc: pt.ForcingData = pt.ForcingData(
    precip=in_df.Precipitation, temp=in_df.Temperature, pet=in_df.PET
)
q_meas: Series = in_df["Qobs"]

In [4]:
# res = model.run(forc=forc, init_state=init_state, streamflow=q_meas)
res = model.run(forc=[forc, forc], init_state=init_state, streamflow=q_meas)

In [5]:
col_names = model.model.column_names

In [5]:
%%prun
res = model.run(forc=forc, init_state=init_state, streamflow=q_meas)

         186700 function calls (186545 primitive calls) in 0.090 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2956    0.015    0.000    0.025    0.000 numeric.py:2337(isclose)
     1460    0.009    0.000    0.103    0.000 hydro.py:70(step)
     1460    0.009    0.000    0.049    0.000 _zeros_py.py:109(newton)
     1460    0.005    0.000    0.059    0.000 _root_scalar.py:62(root_scalar)
      365    0.004    0.000    0.138    0.000 model.py:231(step)
     4416    0.004    0.000    0.008    0.000 hydro.py:95(mass_balance)
     4380    0.003    0.000    0.003    0.000 {built-in method builtins.locals}
    14780    0.002    0.000    0.003    0.000 numeric.py:2429(<genexpr>)
     4416    0.002    0.000    0.010    0.000 hydro.py:80(f)
        1    0.002    0.002    0.082    0.082 model.py:602(run_hydro_model)
    14744    0.002    0.000    0.002    0.000 {built-in method builtins.abs}
16961/16949    0.002    0.000    0.0

In [ ]:
opt_res = model.simple_calibration(
    forc=forc, meas_streamflow=q_meas, metric="kge", polish=True
)

/home/andrew/.local/lib/python3.13/site-packages/scipy/optimize/_differentialevolution.py:487: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


In [9]:
opt_res[1].nse

np.float64(0.8310190183958035)